## 학습코드 (resnet 18 사용)

In [ ]:
import os
import xml.etree.ElementTree as ET
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# 데이터셋의 루트 경로
data_root = r'C:\Users\Home\Desktop\matlab_ai'

# 카테고리 정의
categories = ['alley', 'braille_guide_blocks', 'caution_zone', 'roadway', 'sidewalk']

# Custom Dataset 클래스 정의
class SurfaceDataset(Dataset):
    def __init__(self, root, categories, transforms=None):
        self.root = root
        self.transforms = transforms
        self.categories = categories
        self.images = []
        self.labels = []

        # Surface_001 ~ Surface_300 폴더 목록 가져오기
        dataset_folders = [f'Surface_{str(i).zfill(3)}' for i in range(1, 301)]

        for folder in dataset_folders:
            # 이미지 폴더 경로와 XML 파일 경로 설정
            image_folder = os.path.join(root, folder)

            # XML 파일 목록 가져오기
            xml_files = [f for f in os.listdir(image_folder) if f.endswith('.xml')]

            for xml_file in xml_files:
                # XML 파일 파싱
                xml_path = os.path.join(image_folder, xml_file)
                tree = ET.parse(xml_path)
                root_elem = tree.getroot()

                # 이미지 정보 추출
                for image_elem in root_elem.findall('.//image'):
                    image_name = image_elem.get('name')
                    image_path = os.path.join(image_folder, image_name)
                    
                    if not os.path.exists(image_path):
                        print(f"Image file does not exist: {image_path}")
                        continue

                    # 레이블 추출 (여기서는 첫 번째 폴리곤의 레이블 사용)
                    polygon_elem = image_elem.find('.//polygon')
                    if polygon_elem is None:
                        continue
                    label = polygon_elem.get('label')
                    if label not in self.categories:
                        continue

                    self.images.append(image_path)
                    self.labels.append(self.categories.index(label))

        print(f"Loaded {len(self.images)} images and labels.")

    def __getitem__(self, idx):
        # 이미지 로드
        img_path = self.images[idx]
        label = self.labels[idx]

        img = Image.open(img_path).convert("RGB")
        
        # Transform 수행
        if self.transforms is not None:
            img = self.transforms(img)

        return img, label

    def __len__(self):
        return len(self.images)

# 데이터 변환 정의
transform = T.Compose([
    T.Resize((224, 224)),  # 이미지 크기 조정
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 데이터셋 인스턴스 생성
dataset = SurfaceDataset(data_root, categories, transform)

# 데이터셋을 학습 및 검증 세트로 분할
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# 데이터로더 생성
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)

# 미리 학습된 ResNet18 모델 불러오기
model = models.resnet18(pretrained=True)

# 출력 레이어 수정 (노면 분류 카테고리 수에 맞게)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(categories))

# GPU 사용 가능 여부 확인 및 모델 이동
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 및 검증 함수 정의
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=25):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        # 학습 데이터에 대해 배치 단위로 진행 상황 표시
        for images, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch'):
            images = images.to(device)
            labels = labels.to(device)

            # 이전 gradients를 0으로 설정
            optimizer.zero_grad()

            # forward
            outputs = model(images)
            loss = criterion(outputs, labels)

            # backward
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch {epoch}/{num_epochs-1}, Loss: {epoch_loss:.4f}")

        # 검증
        model.eval()
        val_loss = 0.0
        corrects = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * images.size(0)
                _, preds = torch.max(outputs, 1)
                corrects += torch.sum(preds == labels.data)

        val_loss = val_loss / len(val_loader.dataset)
        val_acc = corrects.double() / len(val_loader.dataset)
        print(f"Validation Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}")

# 모델 학습
train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=25)

## 학습된 가중치 저장 (resnet18 사용)

In [2]:
torch.save(model.state_dict(), 'path_to_save_model.pth')

### 예측 ( 이미지 x ) (resnet50)

In [1]:
import os
import xml.etree.ElementTree as ET
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

transform = T.Compose([
    T.Resize((224, 224)),  # 이미지 크기 조정
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

categories = ['alley', 'braille_guide_blocks', 'caution_zone', 'roadway', 'sidewalk']

model = models.resnet50(pretrained=False)

# 출력 레이어 수정 (노면 분류 카테고리 수에 맞게)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(categories))

# GPU 사용 가능 여부 확인 및 모델 이동
device = torch.device('cuda')
model = model.to(device)

model = torch.load('model_tm.pth')

for i in range(1,6):
    image_path = f'test{i}.jpg'
    img = Image.open(image_path).convert("RGB")
    img_t = transform(img)
    img_t = img_t.unsqueeze(0).to(device)  # 배치 차원 추가 및 GPU로 이동

    # 모델을 사용하여 예측
    with torch.no_grad():
        outputs = model(img_t)
        _, preds = torch.max(outputs, 1)
        pred_class = categories[preds.item()]
        probabilities = torch.nn.functional.softmax(outputs, dim=1)[0] * 100  # 확률 계산
    
    # 예측 결과 출력
    print(f"Predicted class: {pred_class}")
    for i, prob in enumerate(probabilities):
        print(f"{categories[i]}: {prob:.2f}%")
    print()

Predicted class: sidewalk
alley: 4.93%
braille_guide_blocks: 1.02%
caution_zone: 1.38%
roadway: 0.66%
sidewalk: 92.00%

Predicted class: alley
alley: 99.96%
braille_guide_blocks: 0.00%
caution_zone: 0.00%
roadway: 0.02%
sidewalk: 0.02%

Predicted class: alley
alley: 99.20%
braille_guide_blocks: 0.16%
caution_zone: 0.04%
roadway: 0.00%
sidewalk: 0.60%

Predicted class: caution_zone
alley: 0.00%
braille_guide_blocks: 0.00%
caution_zone: 99.89%
roadway: 0.10%
sidewalk: 0.01%

Predicted class: sidewalk
alley: 0.00%
braille_guide_blocks: 0.00%
caution_zone: 0.90%
roadway: 9.04%
sidewalk: 90.06%



## 예측 ( 이미지 x ) (resnet18)

In [9]:
import torch
from PIL import Image
import torchvision.transforms as T
import torchvision.models as models
import torch.nn as nn

# 카테고리 정의
categories = ['alley', 'braille_guide_blocks', 'caution_zone', 'roadway', 'sidewalk']

# 이미지 전처리 파이프라인 정의
transform = T.Compose([
    T.Resize((224, 224)),  # 이미지 크기 조정
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 미리 학습된 ResNet18 모델 불러오기 및 수정된 출력 레이어 설정
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(categories))

# 저장된 모델 가중치 로드 (모델 가중치 파일 경로를 사용하여 로드)
model.load_state_dict(torch.load('path_to_save_model.pth'))

# GPU 사용 가능 여부 확인 및 모델 이동
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()  # 모델을 평가 모드로 설정

# 예측할 이미지 파일 경로
image_path = r'C:\Users\Home\Desktop\matlab_ai\test_data\7.jpg'

# 이미지 열기 및 전처리
img = Image.open(image_path).convert("RGB")
img_t = transform(img)
img_t = img_t.unsqueeze(0).to(device)  # 배치 차원 추가 및 GPU로 이동

# 모델을 사용하여 예측
with torch.no_grad():
    outputs = model(img_t)
    _, preds = torch.max(outputs, 1)
    pred_class = categories[preds.item()]
    probabilities = torch.nn.functional.softmax(outputs, dim=1)[0] * 100  # 확률 계산

# 예측 결과 출력
print(f"Predicted class: {pred_class}")
for i, prob in enumerate(probabilities):
    print(f"{categories[i]}: {prob:.2f}%")

Predicted class: caution_zone
alley: 0.00%
braille_guide_blocks: 0.04%
caution_zone: 47.40%
roadway: 26.04%
sidewalk: 26.52%


# 예측 ( 이미지 )

In [ ]:
# pip install matplotlib
# pip uninstall opencv-python-headless
# pip install opencv-python

### 카메라 사용 with resnet50 

In [1]:
import cv2
import torch
from PIL import Image
import torchvision.transforms as T
import torchvision.models as models
import torch.nn as nn

# 카테고리 정의
categories = ['alley', 'braille_guide_blocks', 'caution_zone', 'roadway', 'sidewalk']

# 이미지 전처리 파이프라인 정의
transform = T.Compose([
    T.Resize((224, 224)),  # 이미지 크기 조정
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# GPU 사용 가능 여부 확인 및 모델 이동
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 초기화 및 가중치 로드
model = models.resnet50(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(categories))
model_weights_path = r'C:\Users\Home\Desktop\matlab_ai\model_50.pth'
model.load_state_dict(torch.load(model_weights_path, map_location=device))
model = model.to(device)
model.eval()  # 모델을 평가 모드로 설정

# 카메라 캡처 설정
cap = cv2.VideoCapture(1)  # 1은 USB 카메라를 의미합니다.

# 확률 임계값 설정 (75%)
threshold = 80.0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # 이미지 전처리
    img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)).convert("RGB")
    img_t = transform(img)
    img_t = img_t.unsqueeze(0).to(device)  # 배치 차원 추가 및 GPU로 이동

    # 모델을 사용하여 예측
    with torch.no_grad():
        outputs = model(img_t)
        _, preds = torch.max(outputs, 1)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)[0] * 100  # 확률 계산
    
    # 확률이 임계값 이상인지 확인
    max_prob = probabilities.max().item()
    if max_prob >= threshold:
        pred_class = categories[preds.item()]
        text = f"Predicted: {pred_class}, Prob: {max_prob:.0f}%"
    else:
        text = "Prediction: Uncertain"

    # 예측 결과를 프레임 위에 표시
    cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # 결과 프레임 출력
    cv2.imshow('Surface Detection', frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


###  (메인으로사용) 카메라 사용 withe resnet50 + tts

In [2]:
!pip install pyttsx3

In [25]:
pip install playsound

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
import torch
from PIL import Image
import torchvision.transforms as T
import torchvision.models as models
import torch.nn as nn
import pyttsx3
import threading
from playsound import playsound

# 카테고리 정의
categories = ['alley', 'braille_guide_blocks', 'caution_zone', 'roadway', 'sidewalk']
# 카테고리 대체 문구 정의
category_descriptions = {
    'alley': '보차혼용도로',
    'braille_guide_blocks': '점자블록',
    'caution_zone': '주의구역',
    'roadway': '차도',
    'sidewalk': '인도'
}

# 이미지 전처리 파이프라인 정의
transform = T.Compose([
    T.Resize((224, 224)),  # 이미지 크기 조정
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# GPU 사용 가능 여부 확인 및 모델 이동
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 초기화 및 가중치 로드
model = models.resnet50(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(categories))
model_weights_path = r'C:\Users\Home\Desktop\matlab_ai\model_50.pth'
model.load_state_dict(torch.load(model_weights_path, map_location=device))
model = model.to(device)
model.eval()  # 모델을 평가 모드로 설정

# TTS 엔진 초기화
engine = pyttsx3.init()

# TTS 기능을 스레드에서 실행하는 함수 정의
def speak(text):
    engine.say(text)
    engine.runAndWait()

# 사운드를 스레드에서 실행하는 함수 정의
def play_sound(file_path):
    playsound(file_path)

# 카메라 캡처 설정
cap = cv2.VideoCapture(1)  # 1은 USB 카메라를 의미합니다.

# 확률 임계값 설정 (95% for alley, 90% for others)
threshold_alley = 95.0
threshold_others = 90.0

# 이전 예측된 카테고리와 TTS 상태 초기화
prev_valid_pred_class = None
tts_spoken = False

# 경고 사운드 파일 경로 설정
warning_sound_path = r'C:\Users\Home\Desktop\matlab_ai\dding.mp3'  # 경고 사운드 파일 경로

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # 이미지 전처리
    img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)).convert("RGB")
    img_t = transform(img)
    img_t = img_t.unsqueeze(0).to(device)  # 배치 차원 추가 및 GPU로 이동

    # 모델을 사용하여 예측
    with torch.no_grad():
        outputs = model(img_t)
        _, preds = torch.max(outputs, 1)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)[0] * 100  # 확률 계산
    
    # 현재 예측된 카테고리와 확률
    pred_class = categories[preds.item()]
    max_prob = probabilities.max().item()
    
    # 해당 카테고리에 대한 적절한 임계값 설정
    if pred_class == 'alley':
        threshold = threshold_alley
    else:
        threshold = threshold_others

    # 확률이 임계값 이상인지 확인
    if max_prob >= threshold:
        text = f"Predicted: {pred_class}, Prob: {max_prob:.0f}%"
        
        # 유효한 예측이 변경된 경우에만 TTS 또는 사운드 실행 (alley <-> sidewalk <-> caution_zone 제외)
        if pred_class != prev_valid_pred_class:
            if pred_class == 'caution_zone':
                sound_thread = threading.Thread(target=play_sound, args=(warning_sound_path,))
                sound_thread.start()
            elif pred_class != 'alley':  # alley는 제외하고 음성 출력
                tts_text = category_descriptions[pred_class]
                tts_thread = threading.Thread(target=speak, args=(tts_text,))
                tts_thread.start()
            prev_valid_pred_class = pred_class
            tts_spoken = True
    else:
        text = "Prediction: Uncertain"

    # 예측 결과를 프레임 위에 표시
    cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # 결과 프레임 출력
    cv2.imshow('Surface Detection', frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import torch
from PIL import Image
import torchvision.transforms as T
import torchvision.models as models
import torch.nn as nn
import pyttsx3
import threading
from playsound import playsound

# 카테고리 정의
categories = ['alley', 'braille_guide_blocks', 'caution_zone', 'roadway', 'sidewalk']
# 카테고리 대체 문구 정의
category_descriptions = {
    'alley': '보차혼용도로',
    'braille_guide_blocks': '점자블록',
    'caution_zone': '주의구역',
    'roadway': '차도',
    'sidewalk': '인도'
}

# 이미지 전처리 파이프라인 정의
transform = T.Compose([
    T.Resize((224, 224)),  # 이미지 크기 조정
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# GPU 사용 가능 여부 확인 및 모델 이동
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 초기화 및 가중치 로드
model = models.resnet50(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(categories))
model_weights_path = r'C:\Users\Home\Desktop\matlab_ai\model_50.pth'
model.load_state_dict(torch.load(model_weights_path, map_location=device))
model = model.to(device)
model.eval()  # 모델을 평가 모드로 설정

# TTS 엔진 초기화
engine = pyttsx3.init()

# TTS 기능을 스레드에서 실행하는 함수 정의
def speak(text):
    engine.say(text)
    engine.runAndWait()

# 사운드를 스레드에서 실행하는 함수 정의
def play_sound(file_path):
    playsound(file_path)

# 카메라 캡처 설정
cap = cv2.VideoCapture(1)  # 1은 USB 카메라를 의미합니다.

# 확률 임계값 설정 (95% for alley, 90% for others)
threshold_alley = 95.0
threshold_others = 90.0

# 이전 예측된 카테고리와 TTS 상태 초기화
prev_valid_pred_class = None
tts_spoken = False
prev_caution_zone = False

# 경고 사운드 파일 경로 설정
warning_sound_path = r'C:\Users\Home\Desktop\matlab_ai\dding.mp3'  # 경고 사운드 파일 경로

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # 이미지 전처리
    img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)).convert("RGB")
    img_t = transform(img)
    img_t = img_t.unsqueeze(0).to(device)  # 배치 차원 추가 및 GPU로 이동

    # 모델을 사용하여 예측
    with torch.no_grad():
        outputs = model(img_t)
        _, preds = torch.max(outputs, 1)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)[0] * 100  # 확률 계산
    
    # 현재 예측된 카테고리와 확률
    pred_class = categories[preds.item()]
    max_prob = probabilities.max().item()
    
    # 해당 카테고리에 대한 적절한 임계값 설정
    if pred_class == 'alley':
        threshold = threshold_alley
    else:
        threshold = threshold_others

    # 확률이 임계값 이상인지 확인
    if max_prob >= threshold:
        text = f"Predicted: {pred_class}, Prob: {max_prob:.0f}%"
        
        # 유효한 예측이 변경된 경우에만 TTS 또는 사운드 실행 (alley <-> sidewalk <-> caution_zone 제외)
        if pred_class == 'caution_zone':
            if not prev_caution_zone:
                sound_thread = threading.Thread(target=play_sound, args=(warning_sound_path,))
                sound_thread.start()
                prev_caution_zone = True
                prev_valid_pred_class = None  # 주의구역 이후에는 음성 출력 초기화
        elif pred_class != prev_valid_pred_class:
            if prev_valid_pred_class in ['alley', 'sidewalk', 'roadway', 'braille_guide_blocks']:
                if pred_class in ['alley', 'sidewalk', 'roadway', 'braille_guide_blocks']:
                    tts_text = category_descriptions[pred_class]
                    tts_thread = threading.Thread(target=speak, args=(tts_text,))
                    tts_thread.start()
            else:
                tts_text = category_descriptions[pred_class]
                tts_thread = threading.Thread(target=speak, args=(tts_text,))
                tts_thread.start()
            prev_valid_pred_class = pred_class
            prev_caution_zone = False  # 주의구역이 아닌 카테고리가 나올 경우 주의구역 상태 초기화
    else:
        text = "Prediction: Uncertain"
        prev_caution_zone = False  # Uncertain 상태일 때 주의구역 상태 초기화

    # 예측 결과를 프레임 위에 표시
    cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # 결과 프레임 출력
    cv2.imshow('Surface Detection', frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


### 카메라 사용 with resnet18

In [13]:
import cv2
import torch
from PIL import Image
import torchvision.transforms as T
import torchvision.models as models
import torch.nn as nn

# 카테고리 정의
categories = ['alley', 'braille_guide_blocks', 'caution_zone', 'roadway', 'sidewalk']

# 이미지 전처리 파이프라인 정의
transform = T.Compose([
    T.Resize((224, 224)),  # 이미지 크기 조정
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# GPU 사용 가능 여부 확인 및 모델 이동
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 초기화 및 가중치 로드
model = models.resnet18(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(categories))
model_weights_path = r'C:\Users\Home\Desktop\matlab_ai\path_to_save_model.pth'
model.load_state_dict(torch.load(model_weights_path, map_location=device))
model = model.to(device)
model.eval()  # 모델을 평가 모드로 설정

# 카메라 캡처 설정
cap = cv2.VideoCapture(1)  # 1은 USB 카메라를 의미합니다.

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # 이미지 전처리
    img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)).convert("RGB")
    img_t = transform(img)
    img_t = img_t.unsqueeze(0).to(device)  # 배치 차원 추가 및 GPU로 이동

    # 모델을 사용하여 예측
    with torch.no_grad():
        outputs = model(img_t)
        _, preds = torch.max(outputs, 1)
        pred_class = categories[preds.item()]
        probabilities = torch.nn.functional.softmax(outputs, dim=1)[0] * 100  # 확률 계산

    # 예측 결과를 프레임 위에 표시
    text = f"Predicted: {pred_class}, Prob: {probabilities.max().item():.0f}%"  # 소수점 없이 표시
    cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # 결과 프레임 출력
    cv2.imshow('Surface Detection', frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
pip install tqdm
pip install tensorflow
pip install albumentations
pip install opencv-python-headless